In [1]:
import pandas as pd

import tensorflow as tf
import keras
import torch

import matplotlib.pyplot as plt
import seaborn as sb

In [6]:
email = pd.read_csv('../../../../../../data/r4.2/email.csv')
file = pd.read_csv('../../../../../../data/r4.2/file.csv')
http = pd.read_csv('../../../../../../data/r4.2/http.csv')
logon = pd.read_csv('../../../../../../data/r4.2/logon.csv')
device = pd.read_csv('../../../../../../data/r4.2/device.csv')


In [5]:

enron = pd.read_csv('../../../../../../data/praxis/ENRON/emails_enron.csv')

In [19]:
files = [file,email,http,logon,device]

In [21]:
for x in files:
    print(x.shape)


(445581, 6)
(2629979, 11)
(28434423, 6)
(854859, 5)
(405380, 5)


In [7]:
enron.shape

(517401, 2)

In [8]:
enron.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...
